In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install tensorflow
!pip install matplotlib

In [ ]:
#pd imports(for reading csv data input)
import pandas as pd
#np js bc
import numpy as np

In [ ]:
#sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
train_data = pd.read_csv('data/california_housing_train.csv').dropna()
test_data = pd.read_csv('data/california_housing_test.csv').dropna()

In [ ]:
train_data.head()

In [ ]:
train_data.tail()

y value is very big:
  
  - need to standardize
  - need to make the mean zero(subtract the mean from all the values)

this can be done using sk-learn's StandardScaler

In [ ]:
scaler = StandardScaler()

X_train = train_data.drop(columns="median_house_value")
y_train = train_data["median_house_value"]

X_test = test_data.drop(columns="median_house_value")
y_test = test_data["median_house_value"]

#standardize data
scaled_X_train = scaler.fit_transform(X_train.values)
scaled_y_train = scaler.fit_transform(y_train.values.reshape(-1, 1))

scaled_X_test = scaler.fit_transform(X_test.values)
scaled_y_test = scaler.fit_transform(y_test.values.reshape(-1, 1))

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input

In [ ]:
#find input
print(X_train.shape[1])

In [ ]:
def create_model():
  model = Sequential();
  model.add(Input(shape=(8,)))
  model.add(Dense(units=64, activation="relu"))
  model.add(Dense(units=32, activation="relu"))
  model.add(Dense(units=1))
  return model

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model=create_model()
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='mean_squared_error',
              metrics=['mae']
              )

In [ ]:
#earlystopping to prevent overfitting
early_stopping = EarlyStopping(patience=5, min_delta=0.0001, verbose=1, restore_best_weights=True)

model.fit(x=scaled_X_train, y=scaled_y_train,
          batch_size=32, epochs = 600,
          callbacks=[early_stopping],
          verbose=True)

In [ ]:
#save to ur folder
!mkdir -p models
model.save(filepath="models/600-epoch.keras")

In [ ]:
#converting scaled predictions into real values
y_hat = scaler.inverse_transform((model.predict(scaled_X_test)))

In [ ]:
y_hat

In [ ]:
from sklearn.metrics import mean_absolute_error

# percentage error
print(mean_absolute_error(y_true=y_test , y_pred=y_hat) / np.mean(y_test))

In [ ]:
#i think u should train w less epoch